# Fashion-MNIST Classification with `ImageDataset`

This notebook demonstrates how to use the `ImageDataset` and `Dataloader` to train a classifier on the Fashion-MNIST dataset.

We will:
1.  Download the Fashion-MNIST dataset using `torchvision`.
2.  Extract the images into a directory structure suitable for `ImageDataset`.
3.  Define image transformations using our library's functions.
4.  Load the data using `ImageDataset` and `Dataloader`.
5.  Build and train a simple MLP classifier.
6.  Use the `Adam` optimizer and `StepLR` scheduler.
7.  Evaluate the model's accuracy and visualize the results.

In [1]:
import sys
sys.path.append('../../../')

import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from PIL import Image

from clownpiece import Tensor
from clownpiece.autograd import no_grad
from clownpiece.nn import Module, Linear, ReLU, Sequential, CrossEntropyLoss
from clownpiece.utils.data.dataset import ImageDataset, sequential_transform, resize_transform, normalize_transform, to_tensor_transform
from clownpiece.utils.data.dataloader import Dataloader
from clownpiece.utils.optim.optimizer import Adam
from clownpiece.utils.optim.lr_scheduler import StepLR

### 1. Download and Extract the Fashion-MNIST Dataset

First, we download the Fashion-MNIST dataset using `torchvision`. Then, we define a helper function `extract_to_folders` to save the images into a directory structure that `ImageDataset` can read. The structure will be `data/<split>/<class_name>/<image_index>.png`.

This extraction process is only performed once. If the directories already exist, this step is skipped.

In [2]:
def extract_to_folders(dataset, root_dir, class_names):
    if os.path.exists(root_dir):
        print(f"'{root_dir}' already exists. Skipping extraction.")
        return
    
    os.makedirs(root_dir)
    for class_name in class_names:
        os.makedirs(os.path.join(root_dir, class_name), exist_ok=True)
        
    for i, (image, label) in enumerate(dataset):
        class_name = class_names[label]
        image_path = os.path.join(root_dir, class_name, f"{i}.png")
        image.save(image_path)
    print(f"Extracted {len(dataset)} images to '{root_dir}'.")

# Download the datasets
print("Downloading Fashion-MNIST...")
train_dataset_raw = datasets.FashionMNIST('./data', train=True, download=True)
test_dataset_raw = datasets.FashionMNIST('./data', train=False, download=True)
print("Download complete.")

class_names = train_dataset_raw.classes

# Extract to folder structure
extract_to_folders(train_dataset_raw, './data/fashion_mnist/train', class_names)
extract_to_folders(test_dataset_raw, './data/fashion_mnist/test', class_names)

100%|██████████| 26.4M/26.4M [00:06<00:00, 4.29MB/s]

100%|██████████| 29.5k/29.5k [00:00<00:00, 148kB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 148kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.19MB/s]

100%|██████████| 5.15k/5.15k [00:00<00:00, 4.49MB/s]



Download complete.
Extracted 60000 images to './data/fashion_mnist/train'.
Extracted 60000 images to './data/fashion_mnist/train'.
Extracted 10000 images to './data/fashion_mnist/test'.
Extracted 10000 images to './data/fashion_mnist/test'.


### 2. Define Transformations and Load Data

Now we define the image transformations using the functions from our library. We'll resize the images, normalize them, and convert them to `clownpiece` Tensors. Then, we create `ImageDataset` and `Dataloader` instances for both the training and test sets.

In [3]:
# Define transformations
img_size = (28, 28)
# For grayscale, mean and std are single values
mean = 0.2860 # Calculated from the training set
std = 0.3530  # Calculated from the training set

transform = sequential_transform(
    resize_transform(img_size),
    normalize_transform(mean, std),
    to_tensor_transform()
)

# Create Datasets
train_dataset = ImageDataset('./data/fashion_mnist/train', transform=transform)
test_dataset = ImageDataset('./data/fashion_mnist/test', transform=transform)

# Create Dataloaders
batch_size = 64
train_loader = Dataloader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = Dataloader(test_dataset, batch_size=1000, shuffle=False)

print(f"Loaded {len(train_dataset)} training samples and {len(test_dataset)} test samples.")

KeyboardInterrupt: 

### 3. Defining the Model Architecture

We'll use a simple Multi-Layer Perceptron (MLP) for this classification task. It consists of two hidden layers with ReLU activations.

In [ ]:
# Define the model
input_features = img_size[0] * img_size[1]
num_classes = len(class_names)

model = Sequential(
    Linear(input_features, 128),
    ReLU(),
    Linear(128, 64),
    ReLU(),
    Linear(64, num_classes)
)

print("Model Architecture:")
print(model)

### 4. Training the Model

We set up the `CrossEntropyLoss` function, the `Adam` optimizer, and a `StepLR` scheduler. The training loop iterates through the `Dataloader`, performs forward and backward passes, and updates the model parameters.

In [ ]:
# Loss, optimizer, and scheduler
loss_fn = CrossEntropyLoss()
init_lr = 1e-3
epochs = 10

optimizer = Adam(model.parameters(), lr=init_lr)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

# Initialize lists to track losses and accuracies
train_losses = []
test_accuracies = []

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    sum_train_loss = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        # Flatten the image data
        X_batch_flat = X_batch.reshape([X_batch.shape[0], -1])
        
        # Forward pass
        logits = model(X_batch_flat)

        # Calculate loss
        loss = loss_fn(logits, y_batch)
        sum_train_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = sum_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Update learning rate
    scheduler.step()
            
    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with no_grad():
        for X_batch, y_batch in test_loader:
            X_batch_flat = X_batch.reshape([X_batch.shape[0], -1])
            logits_test = model(X_batch_flat)
            pred = np.argmax(logits_test.tolist(), axis=1)
            correct += np.sum(pred == np.array(y_batch.tolist()))
            total += y_batch.shape[0]
            
    accuracy = 100. * correct / total
    test_accuracies.append(accuracy)

    print(f"Epoch {epoch+1:2}/{epochs}, train loss: {avg_train_loss:.4f}, test accuracy: {accuracy:.2f}%")

### 5. Visualizing the Results

Finally, we plot the training loss and test accuracy over epochs to assess the model's performance.

In [ ]:
# Visualize Training Loss and Test Accuracy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses, label='Training Loss')
ax1.set_title("Training Loss Over Epochs")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Cross-Entropy Loss")
ax1.grid(True)

ax2.plot(test_accuracies, label='Test Accuracy', color='orange')
ax2.set_title("Test Accuracy Over Epochs")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy (%)")
ax2.set_ylim(0, 100)
ax2.grid(True)

plt.show()